In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Define the data directory where your calisthenics spot images are stored in your Google Drive.
data_dir = "/content/drive/Othercomputers/Meu computador/5-periodo/method"

In [4]:
# Define the image size (adjust according to your model's input size)
img_width, img_height = 224, 224

# Define the batch size
batch_size = 32

# Initialize the ImageDataGenerator for data augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

# Load the data using the ImageDataGenerator
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    classes=['calisthenics_spot']
)

Found 6552 images belonging to 1 classes.


In [28]:

def build_and_train_model():
    # Load the Xception model (excluding the top layer)
    base_model = Xception(weights=None, include_top=False, input_shape=(img_width, img_height, 3))

    # Add a global average pooling layer and a dense output layer for binary classification
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    predictions = Dense(1, activation='sigmoid')(x)

    # Create the final model
    model = Model(inputs=base_model.input, outputs=predictions)

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

    # Define the number of training steps per epoch and the number of epochs
    steps_per_epoch = train_generator.n // train_generator.batch_size
    epochs = 20 # 20

    # Train the model
    model.fit(
        train_generator,
        steps_per_epoch=steps_per_epoch,
        epochs=epochs,
    )

    # Save the model
    model.save("./notes/calisthenics_spot_model.h5")

In [29]:
# Build and train the model
build_and_train_model()

Epoch 1/20
204/204 [==============================] - 187s 751ms/step - loss: 0.0085 - accuracy: 1.0000
Epoch 2/20
204/204 [==============================] - 154s 752ms/step - loss: 1.0048e-05 - accuracy: 1.0000
Epoch 3/20
204/204 [==============================] - 152s 737ms/step - loss: 2.2902e-06 - accuracy: 1.0000
Epoch 4/20
204/204 [==============================] - 150s 736ms/step - loss: 9.9123e-07 - accuracy: 1.0000
Epoch 5/20
 73/204 [=========>....................] - ETA: 1:42 - loss: 6.5496e-07 - accuracy: 1.0000

KeyboardInterrupt: ignored

In [24]:
def test_model(test_image_path):
    # Load the trained model
    model = tf.keras.models.load_model("./notes/calisthenics_spot_model.h5")

    # Load a test image
    test_image = tf.keras.preprocessing.image.load_img(test_image_path, target_size=(img_width, img_height))
    test_image = tf.keras.preprocessing.image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0)
    test_image = test_image / 255.0  # Normalize the image pixel values

    # Make predictions
    predictions = model.predict(test_image)
    print(predictions[0][0])
    if predictions[0][0] >= 0.3:
        class_label = "calisthenics spot"
    else:
        class_label = "not calisthenics spot"

    return class_label


In [27]:
# Test the model on a sample test image (replace 'path_to_test_image.jpg' with the actual image path in your Google Drive)
test_image_path = '/content/drive/Othercomputers/Meu computador/5-periodo/method/calisthenics_spot/teste/nao.png'
result = test_model(test_image_path)
print(f"The image is classified as: {result}")


1/1 [==============================] - 1s 718ms/step
0.22568533
The image is classified as: not calisthenics spot
